In [ ]:
# File for print the prediction from the model

In [5]:
# importing libraries
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
import numpy as np

In [2]:
# Load the model
model = tf.keras.models.load_model('model.keras')

In [3]:
# Load the data from the CSV file
df = pd.read_csv(r"data\cleansing\filled\data.csv")

In [4]:
# Select the crops for prediction
crops = ["Rice", "Wheat", "Atta (Wheat)", "Gram Dal", "Tur/Arhar Dal", "Urad Dal", "Moong Dal", "Masoor Dal", "Sugar", "Milk @", "Groundnut Oil (Packed)", "Mustard Oil (Packed)", "Vanaspati (Packed)", "Soya Oil (Packed)", "Sunflower Oil (Packed)", "Palm Oil (Packed)", "Gur", "Tea Loose", "Salt Pack (Iodised)", "Potato", "Onion", "Tomato"]

In [8]:
look_back=7
# Scale the data to be between 0 and 1
scaler = MinMaxScaler(feature_range=(0, 1))

data = df[crops].values
scaled_data = scaler.fit_transform(data)
last_week_data = df[crops].iloc[-7:].values 
scaled_last_week = scaler.transform(last_week_data) 
reshaped_last_week = np.reshape(scaled_last_week, (1, look_back, len(crops))) 

In [9]:
# Predict
prediction_scaled = model.predict(reshaped_last_week)
prediction = scaler.inverse_transform(prediction_scaled)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 253ms/step


In [10]:
# Create DataFrame for 6/9/2024 predictions
predictions_6_9_2024 = pd.DataFrame(data=prediction, columns=crops)

In [ ]:
predictions_6_9_2024

In [ ]:
# Create a dictionary to store the last known data for each state
last_known_data = {}
for state in df['States/UTs'].unique():
    last_known_data[state] = df[df['States/UTs'] == state].iloc[-1][crops].values

In [ ]:
# Predict for all states using the last known data
for state, state_data in last_known_data.items():
    scaled_state_data = scaler.transform(state_data.reshape(1,-1))
    reshaped_state_data = np.reshape(scaled_state_data, (1, 1, len(crops)))

    state_prediction_scaled = model.predict(reshaped_state_data)
    state_prediction = scaler.inverse_transform(state_prediction_scaled)

    predictions_6_9_2024.loc[state] = state_prediction[0]